In [134]:
import requests, json
from collections import OrderedDict
from operator import getitem
import time
import pandas as pd
from web3 import Web3
import warnings
import struct
import binascii

In [135]:
beacon_chain_contract = Web3.to_checksum_address('0x00000000219ab540356cbb839cbe05303d7705fa')
beacon_chain_deposit_event = '0x649bbc62d0e31342afea4e5cd82d4049e7e1ee912fc0889aa790803be39038c5'
api_key = 'AMBERDATA_API_KEY'

In [136]:
# Replace with your AMBERDATA API KEY
eth_node_url = f'https://rpc.web3api.io/api/v2?x-api-key={api_key}'

In [ ]:
w3 = Web3(Web3.HTTPProvider(eth_node_url))

# Check if connected to Ethereum node
if w3.is_connected:
    print("Connected to Ethereum node")
else:
    print("Not connected to Ethereum node")
    exit(1)

In [138]:
headers = {
    "accept": "application/json",
    "x-amberdata-blockchain-id": "ethereum-mainnet",
    "x-api-key": f'{api_key}'
}

In [139]:
#Fetch Contract ABI: 
abi_url = f'https://web3api.io/api/v2/contracts/{beacon_chain_contract}' 

In [140]:
#Fetching contract address ABI, formating and extracting ABI Only. Will use later for parsing logs. 
abi = requests.request("GET", url=abi_url, headers=headers)
abi = json.loads(abi.text)
abi = abi['payload']['abi']

In [141]:
#Create Web3 Contract Instance: 
myContract = w3.eth.contract(address=beacon_chain_contract, abi=abi)

In [142]:
# Helper function to convert bytes to interger. 
def endian_to_int(endian):
    little_endian_bytes = bytes.fromhex(endian)
    reversed_bytes = little_endian_bytes[::-1]
    integer_value = int.from_bytes(reversed_bytes, byteorder='big')
    return integer_value

In [153]:
#Empyt dict for storing returned data. 
collection = {}
id = 0
#For Pagination
pagination = 0
total_pages = 1000
paginationIncrement = 1

In [ ]:
while pagination <= total_pages:
    #Fetch Contract Events Logs. 
    url = f'https://web3api.io/api/v2/addresses/{beacon_chain_contract}/logs?topic={beacon_chain_deposit_event}&page={pagination}&size=100'
    response = requests.request("GET", url, headers=headers)
    while response.status_code == 429:
        time.sleep(1)
        response = requests.request("GET", url, params=querystring)
    batch_list = json.loads(response.text)["payload"]['records']
    for item in batch_list:
            try: 
                id +=1
                collection[id] = {}
                collection[id]['timestamp'] = item["timestamp"]
                collection[id]['transaction_hash'] = item["transactionHash"]
                collection[id]['block_number'] = item['blockNumber']
                # collection[id]['pub_key'] = (item['data'][6] + item['data'][7]).rstrip('0')
                # adding this line and commenting the above line as the above line will strip 
                # off all the 0s of the public key and the public key can end with a zero as well
                # we know that the length is 95 characterss so this line of code takes care of that
                collection[id]['pub_key'] = (item['data'][6] + item['data'][7])[:96]
                collection[id]['withdrawal_credentials'] = item['data'][9]
                collection[id]['amount'] = endian_to_int(item['data'][11])
                collection[id]['deposit_index'] = endian_to_int(item['data'][17])
                collection[id]['signature'] = item['data'][13]+item['data'][14]+item['data'][15]
            except NameError:
                print(NameError)
            except:
                print("No Data")
    print('Finished page',pagination)
    pagination += paginationIncrement


In [155]:
#updating some pandas default settings
pd.set_option('max_colwidth', None) # show full width of showing cols
pd.set_option("expand_frame_repr", False) # print cols 
pd.set_option('display.float_format', lambda x: f'{x:.3f}') # Format number

In [157]:
df = pd.DataFrame.from_dict(collection,orient='index')

In [158]:
df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')

In [ ]:
df

In [163]:
#Alternative Approach is using the ABI + Web3.py to Process_Receipt - This approach is a bit slower since it request an RPC call. 
# while pagination <= total_pages:
#     #Fetch Contract Events Logs. 
#     url = f'https://web3api.io/api/v2/addresses/{beacon_chain_contract}/logs?topic={beacon_chain_deposit_event}&page={pagination}&size=100'
#     response = requests.request("GET", url, headers=headers)
#     while response.status_code == 429:
#         time.sleep(1)
#         response = requests.request("GET", url, params=querystring)
#     batch_list = json.loads(response.text)["payload"]['records']
#     for item in batch_list:
#         try: 
#             print(item["transactionHash"])
#             receipt = w3.eth.get_transaction_receipt(item["transactionHash"])
#             logs = myContract.events.DepositEvent().process_receipt(receipt)
#             id +=1
#             collection[id] = {}
#             collection[id]['timestamp'] = item["timestamp"]
#             collection[id]['transaction_hash'] = item["transactionHash"]
#             collection[id]['blockNumber'] = logs[0]['blockNumber']
#             collection[id]['amount_raw'] = logs[0]['args']['amount']
#             collection[id]['amount_clean'] = endian_to_int(logs[0]['args']['amount'])
#             collection[id]['index'] = logs[0]['args']['index']
#             collection[id]['signature'] = logs[0]['args']['signature']
#             collection[id]['signature_cleaned'] = binascii.hexlify(logs[0]['args']['signature']).decode('utf-8')
#             collection[id]['withdrawal_credentials'] = logs[0]['args']['withdrawal_credentials']
#             collection[id]['withdrawal_credentials_cleaned'] = binascii.hexlify(logs[0]['args']['withdrawal_credentials']).decode('utf-8')
#         except NameError:
#             print(NameError)
#         except:
#             print("No Events at this TX")
#     print('Finished page',pagination)
#     pagination += paginationIncrement
